In [20]:
import pandas as pd
from itertools import combinations

In [22]:
nvalues = pd.read_csv('../data/nutritional_values.csv')
recipes = pd.read_csv('../data/filtered_recipes_100.csv',sep='|')

In [23]:
df = pd.merge(recipes, nvalues, on='Recipe_id', how='right')

In [29]:
filtros = ['Cena','Entrante','Plato principal','Acompañamiento']
df = df[df['Tipo'].isin(filtros)]

In [30]:
wanted_cols =['Recipe_id','Categoria','energia','grasa','proteina','carbohidratos']
df = df[wanted_cols]
cc = list(combinations(df.Recipe_id,2)) #antes de esto hacer filtradosde comida y cena etc
df.set_index('Recipe_id',inplace=True)
df

,Categoria,energia,grasa,proteina,carbohidratos
Recipe_id,,,,,
51,Recetas de Mariscos,686.782370,61.393619,17.454238,14.201002
53,Recetas de Verduras,166.307500,15.307950,2.907813,4.631500
61,Recetas de Mariscos,336.669995,5.152300,48.811874,12.279333
68,Recetas de Aperitivos y tapas,1613.028902,107.064417,50.325723,102.576002
6320,Recetas de Aves y caza,683.579012,21.922922,50.040902,69.503749
...,...,...,...,...,...
74157,Recetas de Carne,3441.640000,266.579000,182.518000,80.731200
74163,Recetas de Arroces y cereales,1482.080000,56.634200,66.497500,173.000000
74164,Recetas de Pescado,923.855879,60.862941,53.324880,40.569605


In [31]:
cc_cols = ['Recipe1','Recipe2'] #poner comida/cena whatever

combined_df = pd.DataFrame(columns = cc_cols)
combined_df['Recipe1'] = [c[0] for c in cc]
combined_df['Recipe2'] = [c[1] for c in cc]
combined_df

,Recipe1,Recipe2
0,51,53
1,51,61
2,51,68
3,51,6320
4,51,6388
...,...,...
4441685,74163,74194
4441686,74163,74200
4441687,74164,74194
4441688,74164,74200


In [33]:
macros = ['energia','grasa','proteina','carbohidratos']
for macro in macros:
    combined_df[macro] = [df[macro][c[1]]+df[macro][c[0]] for c in cc]
combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,51,53,853.089870,76.701569,20.362051,18.832502
1,51,61,1023.452365,66.545919,66.266112,26.480335
2,51,68,2299.811272,168.458036,67.779961,116.777004
3,51,6320,1370.361382,83.316541,67.495140,83.704751
4,51,6388,862.652370,63.757939,20.682188,48.968202
...,...,...,...,...,...,...
4441685,74163,74194,2677.338902,118.489767,193.165848,205.539502
4441686,74163,74200,1824.970000,58.651200,99.565500,219.612000
4441687,74164,74194,2119.114781,122.718508,179.993228,73.109107
4441688,74164,74200,1266.745879,62.879941,86.392880,87.181605


In [40]:
combined_df = combined_df[combined_df['energia']<2500]

In [41]:
combined_df.to_csv('../data/combined.csv', index=False)